# Min Cost Flow ILP

### Author: [Ben Rosenberg](https://www.youtube.com/@benjaminrosenberg)


### Imports

In [6]:
import gurobipy as gp
from gurobipy import GRB
import time

### Input data

In [28]:
nodes = range(4)

edges = [(0,1), (0,3), (2,1), (2,3)]

# cost[i,j] is the cost of sending one unit of flow along edge (i,j)
cost = {
    (0,1) : 0,
    (0,3) : 4,
    (2,1) : 2,
    (2,3) : 8
} 

# capacity[i,j] is the capacity of edge (i,j)
capacity = {
    (0,1) : 5,
    (0,3) : 2,
    (2,1) : 3,
    (2,3) : 2
}

# supply[i] is the supply (if positive) or demand (if negative) of node i
supply = [5, -6, 3, -2]

### Model Definition

In [29]:
start_time = time.time()

model = gp.Model()

# set output level to max
model.Params.TuneOutput = 3

# add variable f
f = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, name='f')

# add constraint representing supply/demand
for i in nodes:
    model.addConstr(gp.quicksum(f[i,j] for (x,j) in edges if x == i) -
                    gp.quicksum(f[j,i] for (j,x) in edges if x == i)
                   == supply[i])

# add constraint on edge flows w.r.t. capacities
for (i,j) in edges:
    model.addConstr(f[i,j] <= capacity[i,j])

# add constraint on edge flows w.r.t. 0
for (i,j) in edges:
    model.addConstr(f[i,j] >= 0)
    
# set objective
model.setObjective(gp.quicksum(cost[i,j] * f[i,j] for (i,j) in edges), GRB.MINIMIZE)

model.optimize()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

try:
    print(f'\nObjective value found: {model.objVal}')
except AttributeError as e:
    print(f'\nCould not find an objective value. \nTraceback:\n\t{e}')


Set parameter TuneOutput to value 3
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12 rows, 16 columns and 16 nonzeros
Model fingerprint: 0xaf2c58bd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 12 rows and 16 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.400000000e+01

[Total time used: 0 minutes, 0 seconds]

Objective value found: 14.0


In [30]:
# optimal solution
for (i,j) in edges:
    print('f[{},{}] = {}'.format(
        i, j, f[i,j].x
    ))

f[0,1] = 3.0
f[0,3] = 2.0
f[2,1] = 3.0
f[2,3] = 0.0
